<a href="https://colab.research.google.com/github/luizamelchor/chatbot_gemini/blob/main/Alura_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
!pip -q install google-adk

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

import google.generativeai as genai

# client = genai.Client()

MODEL_ID = "gemini-pro"

In [ ]:
# Importação de biliotecas de linguagem natural
import spacy
import nltk

# Importação de bibliotecas de geração de texto e tradução
import transformers

# Importação de outras bibliotecas úteis
import random

In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
################### AGENTE 1 ###################
def agente_conversacao(topico):
  conversacao = Agent(
      name='agente_conversacao',
      model='gemini-2.0-flash',
      instruction="""
        Você é um amigo para o usuário que está aprendendo alemão. Seja simpático e encorajador.
        Sua principal função é manter uma conversa descontraída em alemão com o usuário, adaptando o nível de
         dificuldade ao nível de proficiência dele.
        **Na sua primeira interação, pergunte ao usuário sobre o seu nível de alemão (básico, intermediário ou
        avançado) para que você possa começar a conversar em um nível adequado.**
        Responda em alemão e português até que o usuário diga explicitamente que só quer que você use alemão.
        Pergunte ao usuário se ele tem alguma dúvida sobre termos durante a conversa.
        Se o usuário pedir ajuda com gramática, vocabulário ou curiosidades, sugira educadamente que ele encerre
        a conversa e volte ao menu principal para escolher a opção correspondente.

      """,
      description='Agente que conversa com o usuário em Alemão, de acordo com o nível do mesmo'
  )
  return conversacao # Retorna o objeto Agent

In [ ]:
################### Agente 2 ###################
def agente_gramatica(topico):
  gramatica = Agent(
      name='agente_gramatica',
      model='gemini-2.5-pro-exp-03-25',
      instruction="""
        Você é um amigo para o usuário que está aprendendo alemão. Seja simpático e encorajador.
        Sua principal função é explicar de forma descontraída e simples regras gramaticais em alemão para o usuário, adaptando o nível de
        dificuldade ao nível de proficiência dele.
        Se o usuário errar alguma coisa, corrijá-lo de forma leve e simples.
        Não use jargões de linguística, ao menos que o usuário pergunte por algo mais específico.
        Responda em alemão e português até que o usuário diga explicitamente que só quer que você use alemão.
      """,
      description='Agente que explica regras gramaticais do alemão, de acordo com o nível do mesmo'
  )
  return gramatica # Retorna o objeto Agent

In [ ]:
################### Agente 3 ###################
def agente_vocabulario(topico):
  vocabulario = Agent(
      name='agente_vocabulario',
      model='gemini-2.0-flash',
      instruction="""
        Você é um amigo para o usuário que está aprendendo alemão. Seja simpático e encorajador.
        Sua principal função é apresentar de forma descontraída e simples palavras e expressões em alemão para o usuário de acordo com o nível de
        dificuldade dele e escolha de categorias.
      """,
      description='Agente que apresenta novas palavras para o usuário expandir seu conhecimento em alemão.'
  )
  return vocabulario # Retorna o objeto Agent

In [ ]:
################### Agente 4 ###################
def agente_curiosidades(topico):
  curiosidades = Agent(
      name='agente_curiosidades',
      model='gemini-2.0-flash',
      instruction= """
      Você é um grande conhecedor dos países/regiões que falam alemão como língua
      oficial e não oficial. Isto inclui: Alemanha, Suíça, Austria, Bélgica, Luxemburgo,
      Liechtenstein, Südtirol, Oppeln e Voivodia da Silésia.
      Conte curiosidades que o usuário provavelmente não saiba sobre esses lugares e como
      a língua muda de acordo com a região que é falada.
      Conte curiosidades sobre os diferentes sotaques, regras gramáticas, regras ortográficas,
      entre outros tópicos para o usuário.
      """,
      description='Agente que é um grande entusiasta da língua alemã nas diversas localidades que ela está presente'
  )
  return curiosidades # Retorna o objeto Agent

In [ ]:
# As listas de exercícios como exercicios_casos, exercicios_verbos_regulares, etc
exercicios_casos = [
    {
        "pergunta": "Complete a frase: Der Mann gibt ___ Frau einen Blumenstrauß. (a)",
        "resposta": "der"
    },
    {
        "pergunta": "Escolha o caso correto para o pronome: Ich helfe ___ (du).",
        "opcoes": ["dich", "dir", "dein"],
        "resposta": "dir"
    },
    {
        "pergunta": "Reorganize as palavras: dem, Haus, in, wohnt, Er.",
        "resposta": "Er wohnt in dem Haus."
    },
    {
        "pergunta": "Reorganize as palavras: dem, Haus, in, wohnt, Er.",
        "resposta": "Er wohnt in dem Haus."
    },
    {
        "pergunta": "Qual é o caso do substantivo “dem Haus” na frase anterior?",
        "opcoes": ["Nominativ", "Akkusativ", "Dativ", "Genitiv"],
        "resposta": "Dativ"
    },
    {
        "pergunta": "Complete: ___ (die) Katze schläft auf dem Sofa.",
        "resposta": "Die"
    }
]

exercicios_verbos_regulares = [
    {"pergunta": "Conjugue 'machen' na primeira pessoa do singular (ich):", "resposta": "mache"},
    {"pergunta": "Complete: Wir ___ (spielen) Fußball.", "resposta": "spielen"},
    {"pergunta": "Qual a terminação correta para a terceira pessoa do singular (-er) do verbo 'fragen' no Präsens?", "resposta": "t"},
    {"pergunta": "Conjugue 'lernen' na segunda pessoa do plural (ihr) no Präsens:", "resposta": "lernt"},
    {"pergunta": "Traduza para o alemão: 'Eu pergunto'. (usando 'fragen')", "resposta": "Ich frage"}
]

exercicios_verbos_irregulares = [
    {"pergunta": "Conjugue 'sprechen' na segunda pessoa do singular (du) no Präsens:", "resposta": "sprichst"},
    {"pergunta": "Complete a frase: Sie ___ (lesen) ein Buch.", "resposta": "liest"},
    {"pergunta": "Qual a mudança de vogal ocorre no verbo 'fahren' na segunda pessoa do singular (du) no Präsens?", "resposta": "ä"},
    {"pergunta": "Conjugue 'sehen' na terceira pessoa do plural (sie - eles/elas) no Präsens:", "resposta": "sehen"},
    {"pergunta": "Traduza para o alemão: 'Ele come'. (usando 'essen')", "resposta": "Er isst"}
]

exercicios_artigos = [
    {"pergunta": "Qual é o artigo definido nominativo masculino?", "resposta": "der"},
    {"pergunta": "Complete a frase no Akkusativ: Ich sehe ___ (ein) Mann.", "resposta": "einen"},
    {"pergunta": "Qual é o artigo indefinido dativo feminino?", "resposta": "einer"},
    {"pergunta": "Complete a frase no Nominativ plural: ___ (die) Kinder spielen.", "resposta": "Die"},
    {"pergunta": "Qual artigo (definido ou indefinido) usar para 'um livro qualquer' no Akkusativ?", "resposta": "einen"}
]

exercicios_ordem_palavras_principal = [
    {"pergunta": "Reorganize: heute, Ich, ins Kino, gehe.", "resposta": "Ich gehe heute ins Kino."},
    {"pergunta": "Complete: ___ (Er) liest ein Buch.", "resposta": "Er"},
    {"pergunta": "Onde geralmente o verbo conjugado se encontra em uma frase principal?", "resposta": "zweite Position"},
    {"pergunta": "Reorganize: den, Lehrer, fragt, der Schüler.", "resposta": "Der Schüler fragt den Lehrer."},
    {"pergunta": "Traduza para o alemão (atenção à ordem): 'Ela bebe café de manhã'.", "resposta": "Sie trinkt Kaffee am Morgen."}
]

exercicios_ordem_palavras_subordinada = [
    {"pergunta": "Complete a conjunção: Ich weiß, ___ du müde bist.", "resposta": "dass"},
    {"pergunta": "Onde o verbo conjugado se encontra geralmente em uma oração subordinada?", "resposta": "am Ende"},
    {"pergunta": "Reorganize: weil, hat, keine Zeit, er.", "resposta": "weil er keine Zeit hat"},
    {"pergunta": "Qual conjunção subordinada significa 'quando'?", "resposta": "wenn"},
    {"pergunta": "Complete: Er fragt, ob ___ (wir) kommen.", "resposta": "wir kommen"}
]

In [ ]:
################### FUNÇÕES ###################

# Crie uma instância do SessionService e Runner
session_service = InMemorySessionService()
runner = Runner(
    session_service=session_service,
    app_name="GermanTutorApp",
    agents=[
        agente_conversacao,
        agente_gramatica,
        agente_vocabulario,
        agente_curiosidades
    ]
)

# Formatação de texto para markdown
def md(texto):
  display(Markdown(texto))

# Exibição do menu principal
def exibir_menu():
  md('**Você quer aprender sobre qual tópico hoje?**')
  md('1. **Conversação:** Aprenda frases úteis para o dia a dia')
  md('2. **Gramática:** Entenda as regras básicas da língua')
  md('3. **Vocabulário:** Descubra novas palavras e expressões')
  md('4. **Curiosidades:** Explore a cultura alemã!')
  md('**Por favor, digite o número da sua escolha:**')

# Escolha da opção 1: Conversação
def lidar_conversacao():
    md('## 💬 Conversando em Alemão')
    primeira_mensagem_agente = 'Hallo! Antes de começarmos, me diga o seu nível de proficiência em alemão (básico, intermediário ou avançado)? '
    md(f"**Agente:** {primeira_mensagem_agente}")
    historico_chat = []
    model = genai.GenerativeModel('gemini-2.0-flash')
    historico_chat.append({'role': 'model', 'parts': [primeira_mensagem_agente]})
    while True:
        pergunta_usuario = input("Você: ")
        historico_chat.append({'role': 'user', 'parts': [pergunta_usuario]})

        response = model.generate_content(historico_chat)
        resposta_agente = response.text
        md(f"**Agente:** {resposta_agente}")
        historico_chat.append({'role': 'model', 'parts': [resposta_agente]})

        if pergunta_usuario.lower() == "encerrar conversa":
            md("Foi bom conversar com você! 👋")
            break

# Escolha da opção 2: Gramática
def lidar_gramatica():
    md('## ✏️ Aprendendo a gramática Alemã')
    topicos_gramatica = {
      1: {"nome": "Os Casos", "exercicios": exercicios_casos},
      2: {"nome": "Conjugação de Verbos Regulares", "exercicios": exercicios_verbos_regulares},
      3: {"nome": "Conjugação de Verbos Irregulares (Fortes)", "exercicios": exercicios_verbos_irregulares},
      4: {"nome": "Artigos Definidos e Indefinidos", "exercicios": exercicios_artigos},
      5: {"nome": "Ordem das Palavras na Frase Principal", "exercicios": exercicios_ordem_palavras_principal},
      6: {"nome": "Ordem das Palavras na Oração Subordinada", "exercicios": exercicios_ordem_palavras_subordinada},
    }
    md("Escolha um tópico gramatical para estudar:")
    for numero, topico in topicos_gramatica.items():
        md(f"{numero}. **{topico['nome']}**")
    md("**Digite o número do tópico desejado ou 'voltar' para o menu:**")

    while True:
        escolha_topico = input("Você: ").lower()
        if escolha_topico.isdigit() and int(escolha_topico) in topicos_gramatica:
            topico_escolhido = topicos_gramatica[int(escolha_topico)]
            if topico_escolhido["exercicios"]:
                exercicio = random.choice(topico_escolhido["exercicios"])
                md(f"### {topico_escolhido['nome']}")
                md(exercicio["pergunta"])
                resposta_usuario = input("Sua resposta: ")
                 # Lógica modificada para aceitar "ss" como "ß"
                resposta_correta_normalizada = exercicio["resposta"].lower().replace("ß", "ss")
                resposta_usuario_normalizada = resposta_usuario.lower().replace("ß", "ss")

                if resposta_usuario.lower() == exercicio["resposta"].lower():
                    md("**Correto! 👍**")
                else:
                    md(f"**Incorreto.** 😔 A resposta correta é: **{exercicio['resposta']}** (Lembre-se que você pode usar 'ss' no lugar de 'ß')")
            else:
                md("Desculpe, ainda não há exercícios para este tópico.")
        elif escolha_topico == 'voltar':
            break
        else:
            md('**Opção inválida.** Por favor, digite o número do tópico ou "voltar".')
    while True:
        escolha_gramatica = input("Você: ").lower()

        if escolha_gramatica == '1':
            md("## Os Casos (Nominativo, Acusativo, Dativo, Genitivo)")

            while True:
              if exercicios_casos: # Verifica se há exercícios disponíveis
                  exercicio = random.choice(exercicios_casos)
                  md(exercicio["pergunta"])
                  if "opcoes" in exercicio:
                    md(f"Opções: {', '.join(exercicio['opcoes'])}")
                  resposta_usuario = input("Sua resposta: ")
                  resposta_correta_normalizada = exercicio["resposta"].lower().replace("ß", "ss")
                  resposta_usuario_normalizada = resposta_usuario.lower().replace("ß", "ss")

                  if resposta_usuario_normalizada == resposta_correta_normalizada:
                    md("**Correto! 👍**")
                  else:
                    md(f"**Incorreto.** 😔 A resposta correta é: **{exercicio['resposta']}** (Lembre-se que você pode usar 'ss' no lugar de 'ß')")
              else:
                md("Desculpe, não há mais exercícios disponíveis.")
                break

        elif escolha_gramatica == '2':
            md("## Conjugação de Verbos Regulares")

            while True:
              if exercicios_casos: # Verifica se há exercícios disponíveis
                  exercicio = random.choice(exercicios_verbos_regulares)
                  md(exercicio["pergunta"])
                  if "opcoes" in exercicio:
                    md(f"Opções: {', '.join(exercicio['opcoes'])}")
                  resposta_usuario = input("Sua resposta: ")
                  resposta_correta_normalizada = exercicio["resposta"].lower().replace("ß", "ss")
                  resposta_usuario_normalizada = resposta_usuario.lower().replace("ß", "ss")

                  if resposta_usuario_normalizada == resposta_correta_normalizada:
                    md("**Correto! 👍**")
                  else:
                    md(f"**Incorreto.** 😔 A resposta correta é: **{exercicio['resposta']}** (Lembre-se que você pode usar 'ss' no lugar de 'ß')")
              else:
                md("Desculpe, não há mais exercícios disponíveis.")
                break

        elif escolha_gramatica == '3':
           md("## Conjugação de Verbos Irregulares (Fortes)")

           while True:
              if exercicios_casos: # Verifica se há exercícios disponíveis
                  exercicio = random.choice(exercicios_verbos_irregulares)
                  md(exercicio["pergunta"])
                  if "opcoes" in exercicio:
                    md(f"Opções: {', '.join(exercicio['opcoes'])}")
                  resposta_usuario = input("Sua resposta: ")
                  resposta_correta_normalizada = exercicio["resposta"].lower().replace("ß", "ss")
                  resposta_usuario_normalizada = resposta_usuario.lower().replace("ß", "ss")

                  if resposta_usuario_normalizada == resposta_correta_normalizada:
                    md("**Correto! 👍**")
                  else:
                    md(f"**Incorreto.** 😔 A resposta correta é: **{exercicio['resposta']}** (Lembre-se que você pode usar 'ss' no lugar de 'ß')")
              else:
                md("Desculpe, não há mais exercícios disponíveis.")
                break

        elif escolha_gramatica == '4':
            md("## Artigos Definidos e Indefinidos")

            while True:
              if exercicios_casos: # Verifica se há exercícios disponíveis
                  exercicio = random.choice(exercicios_artigos)
                  md(exercicio["pergunta"])
                  if "opcoes" in exercicio:
                    md(f"Opções: {', '.join(exercicio['opcoes'])}")
                  resposta_usuario = input("Sua resposta: ")
                  resposta_correta_normalizada = exercicio["resposta"].lower().replace("ß", "ss")
                  resposta_usuario_normalizada = resposta_usuario.lower().replace("ß", "ss")

                  if resposta_usuario_normalizada == resposta_correta_normalizada:
                    md("**Correto! 👍**")
                  else:
                    md(f"**Incorreto.** 😔 A resposta correta é: **{exercicio['resposta']}** (Lembre-se que você pode usar 'ss' no lugar de 'ß')")
              else:
                md("Desculpe, não há mais exercícios disponíveis.")
                break

        elif escolha_gramatica == '5':
            md("## Ordem das Palavras na Frase Principal")

            while True:
              if exercicios_casos: # Verifica se há exercícios disponíveis
                  exercicio = random.choice(exercicios_ordem_palavras_principal)
                  md(exercicio["pergunta"])
                  if "opcoes" in exercicio:
                    md(f"Opções: {', '.join(exercicio['opcoes'])}")
                  resposta_usuario = input("Sua resposta: ")
                  resposta_correta_normalizada = exercicio["resposta"].lower().replace("ß", "ss")
                  resposta_usuario_normalizada = resposta_usuario.lower().replace("ß", "ss")

                  if resposta_usuario_normalizada == resposta_correta_normalizada:
                    md("**Correto! 👍**")
                  else:
                    md(f"**Incorreto.** 😔 A resposta correta é: **{exercicio['resposta']}** (Lembre-se que você pode usar 'ss' no lugar de 'ß')")
              else:
                md("Desculpe, não há mais exercícios disponíveis.")
                break

        elif escolha_gramatica == '6':
            md("## Ordem das Palavras na Oração Subordinada")

            while True:
              if exercicios_casos: # Verifica se há exercícios disponíveis
                  exercicio = random.choice(exercicios_ordem_palavras_subordinada)
                  md(exercicio["pergunta"])
                  if "opcoes" in exercicio:
                    md(f"Opções: {', '.join(exercicio['opcoes'])}")
                  resposta_usuario = input("Sua resposta: ")
                  resposta_correta_normalizada = exercicio["resposta"].lower().replace("ß", "ss")
                  resposta_usuario_normalizada = resposta_usuario.lower().replace("ß", "ss")

                  if resposta_usuario_normalizada == resposta_correta_normalizada:
                    md("**Correto! 👍**")
                  else:
                    md(f"**Incorreto.** 😔 A resposta correta é: **{exercicio['resposta']}** (Lembre-se que você pode usar 'ss' no lugar de 'ß')")
              else:
                md("Desculpe, não há mais exercícios disponíveis.")
                break
        elif escolha_gramatica == 'voltar':
            md('Voltando ao menu principal...')
            break
        else:
            md('**Opção inválida.** Por favor, digite um número entre 1 e 6.')

# Escolha da opção 3: Vocabulário
def lidar_vocabulario():
    md('## 📖 Vocabulário')
    md('1. **Palavras mágicas**')
    md('2. **Frases indispensáveis**')
    md('3. **Saudações**')
    md('4. **Como se apresentar**')
    md('5. **Verbos fracos, fortes e mistos**')
    md('6. **Declinações**')
    md('7. **Formas do plural**')
    md('8. **Adjetivos possessivos e numéricos**')
    md('9. **Referências espaciais e temporais**')
    md('10. **Substantivos**')
    md('11. **Pronomes**')
    md('12. **Adjetivos**')
    md('13. **Advérbios**')
    md('14. **Preposições**')
    md('15. **Conjunções**')
    md('**Por favor, digite o número da sua escolha:**')

    while True:
        escolha_vocabulario = input("Você: ")
        if escolha_vocabulario.isdigit() and 1 <= int(escolha_vocabulario) <= 15:
          topicos = {
              1: "Palavras mágicas", 2: "Frases indispensáveis", 3: "Saudações",
              4: "Como se apresentar", 5: "Verbos fracos, fortes e mistos", 6: "Declinações",
              7: "Formas do plural", 8: "Adjetivos possessivos e numéricos", 9: "Referências espaciais e temporais",
              10: "Substantivos", 11: "Pronomes", 12: "Adjetivos", 13: "Advérbios",
              14: "Preposições", 15: "Conjunções"
            }
          topico_escolhido_nome = topicos.get(int(escolha_vocabulario), "Tópico de vocabulário genérico")
          mensagem_para_agente = f"Explique sobre: {topico_escolhido_nome}"
          response_iterator = runner.query(query=mensagem_para_agente, agent_name='agente_vocabulario')
          for response in response_iterator:
                 md(f"**Agente Vocabulário:** {response.text}")

        elif escolha_vocabulario.lower() == "voltar":
            md("Voltando ao menu principal...")
            break
        else:
            md("**Opção inválida.** Por favor, digite um número entre 1 e 15.")


# Escolha da opção 4: Curiosidades
def lidar_curiosidades():
      md('## 🥨 Curiosidades')
      md('1. **Dialetos Vivos**')
      md('2. **Cultura na Língua**')
      md('3. **Evolução Curiosa**')
      md('4. **Influências Estrangeiras**')
      md('5. **Fatos Regionais**')
      md('**Por favor, digite o número da sua escolha:**')

      while True:
        escolha_curiosidades = input("Você: ")
      if escolha_curiosidades.isdigit() and 1 <= int(escolha_curiosidades) <= 5:
        topicos_curiosidades = {
            1: "Dialetos Vivos", 2: "Cultura na Língua", 3: "Evolução Curiosa",
            4: "Influências Estrangeiras", 5: "Fatos Regionais"
        }
        topico_escolhido_nome = topicos_curiosidades.get(int(escolha_curiosidades), "Curiosidade geral")
        mensagem_para_agente = f"Conte uma curiosidade sobre: {topico_escolhido_nome}"
        response_iterator = runner.query(query=mensagem_para_agente, agent_name='agente_curiosidades')
        for response in response_iterator:
          md(f"**Agente Curiosidades:** {response.text}")

      elif escolha_curiosidades.lower() == "voltar":
          md("Voltando ao menu principal...")
          break
      else:
          md("**Opção inválida.** Por favor, digite um número entre 1 e 5.")

def obter_escolha():
    while True:
        try:
            escolha = int(input('Digite um número da sua escolha: '))
            if 1 <= escolha <= 4:
                return escolha
            else:
                md('**Opção inválida.** Por favor, digite um número entre 1 e 4.')
        except ValueError:
            md('**Entrada inválida.** Por favor, digite um número.')

TypeError: Runner.__init__() got an unexpected keyword argument 'agents'

  elif escolha_vocabulario == 1:
            md('**Palavras mágicas**')
        elif escolha_vocabulario == 2:
            md('**Frases indispensáveis**')
        elif escolha_vocabulario == 3:
            md('**Saudações**')
        elif escolha_vocabulario == 4:
            md('**Como se apresentar**')
        elif escolha_vocabulario == 5:
            md('**Verbos fracos, fortes e mistos**')
        elif escolha_vocabulario == 6:
          md('**Declinações**')
        elif escolha_vocabulario == 7:
          md('**Formas do plural**')
        elif escolha_vocabulario == 8:
          md('**Adjetivos possessivos e numéricos**')
        elif escolha_vocabulario == 9:
          md('**Referências espaciais e temporais**')
        elif escolha_vocabulario == 10:
          md('**Substantivos**')
        elif escolha_vocabulario == 11:
          md('**Pronomes**')
        elif escolha_vocabulario == 12:
          md('**Adjetivos**')
        elif escolha_vocabulario == 13:
          md('**Advérbios**')
        elif escolha_vocabulario == 14:
          md('**Preposições**')
        elif escolha_vocabulario == 15:
          md('**Conjunções**')


          elif escolha_curiosidades == 1:
          md('**Dialetos Vivos**')
      elif escolha_curiosidades == 2:
          md('**Cultura na Língua**')
      elif escolha_curiosidades == 3:
          md('**Evolução Curiosa**')
      elif escolha_curiosidades == 4:
          md('**Influências Estrangeiras**')
      elif escolha_curiosidades == 5:
          md('**Fatos Regionais**')

## ❗**Programa Principal** ❗

In [ ]:
md('Olá! 👋 Bem vindo ao seu tutor de Alemão!')

while True:
    exibir_menu()
    escolha = obter_escolha()

    if escolha == 1:
        lidar_conversacao()
    elif escolha == 2:
        lidar_gramatica()
    elif escolha == 3:
        lidar_vocabulario()
    elif escolha == 4:
        lidar_curiosidades()

    continuar = input("Deseja aprender mais? (sim/não): ").lower()
    if continuar != 'sim':
        md('Auf Wiedersehen! 👋')
        break

Olá! 👋 Bem vindo ao seu tutor de Alemão!

**Você quer aprender sobre qual tópico hoje?**

1. **Conversação:** Aprenda frases úteis para o dia a dia

2. **Gramática:** Entenda as regras básicas da língua

3. **Vocabulário:** Descubra novas palavras e expressões

...

14. **Preposições**

15. **Conjunções**

**Por favor, digite o número da sua escolha:**

Você: 12


AttributeError: 'LlmAgent' object has no attribute 'send_message'